dependencies and imports

In [70]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

getting data

In [71]:
train_dir = r"C:\Users\benja\Downloads\HackaTUM_Data\HackaTUM_Data\dataset\hackatum_dataset\train"
val_dir = r"C:\Users\benja\Downloads\HackaTUM_Data\HackaTUM_Data\dataset\hackatum_dataset\val"
IMG_SIZE = (256,256)
train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir, shuffle= True, batch_size = 16, image_size = IMG_SIZE)
validation_dataset = tf.keras.utils.image_dataset_from_directory(val_dir, shuffle =True, batch_size = 16, image_size = IMG_SIZE)

Found 500 files belonging to 2 classes.
Found 125 files belonging to 2 classes.


generate test set

In [72]:
val_batches = tf.data.experimental.cardinality(validation_dataset)
test_dataset = validation_dataset.take(val_batches // 4)
validation_dataset = validation_dataset.skip(val_batches // 4)
print('Number of validation batches: %d' % tf.data.experimental.cardinality(validation_dataset))
print('Number of test batches: %d' % tf.data.experimental.cardinality(test_dataset))

Number of validation batches: 6
Number of test batches: 2


configuration of dataset

In [73]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

data augmentation for increase of dataset

In [74]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.1),
])

rescale images to feed into the pretrained model   

In [75]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [76]:
rescale = tf.keras.layers.Rescaling(1./127.5, offset= -1)

create base model --> MobileNet V2

In [77]:
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [78]:
image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

(16, 8, 8, 1280)


freeze weights of pretrained model --> make the upper layers trainable 

In [79]:
base_model.trainable = True

In [80]:
base_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 128, 128, 32  864         ['input_10[0][0]']               
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 128, 128, 32  128         ['Conv1[0][0]']                  
                                )                                              

average features 

In [81]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

(16, 1280)


add dense layer to make prediction 

In [82]:
prediction_layer = tf.keras.layers.Dense(1)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

(16, 1)


stitch the model together 

In [83]:
inputs = tf.keras.Input(shape=(256, 256, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

compile model 

In [84]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [85]:
model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 256, 256, 3)]     0         
                                                                 
 sequential_3 (Sequential)   (None, 256, 256, 3)       0         
                                                                 
 tf.math.truediv_6 (TFOpLamb  (None, 256, 256, 3)      0         
 da)                                                             
                                                                 
 tf.math.subtract_6 (TFOpLam  (None, 256, 256, 3)      0         
 bda)                                                            
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 8, 8, 1280)       2257984   
 ional)                                                          
                                                           

In [86]:
len(model.trainable_variables)

2

In [ ]:
print("Number of layers in the base model: ", len(base_model.layers))


fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

train model 

In [87]:
initial_epochs = 12

loss0, accuracy0 = model.evaluate(validation_dataset)

6/6 [==============================] - 10s 516ms/step - loss: 0.5841 - accuracy: 0.6129


In [89]:
history = model.fit(train_dataset,
                    epochs=initial_epochs,
                    validation_data=validation_dataset)

Epoch 1/10
32/32 [==============================] - 41s 973ms/step - loss: 0.6236 - accuracy: 0.6120 - val_loss: 0.5593 - val_accuracy: 0.6989
Epoch 2/10
32/32 [==============================] - 34s 949ms/step - loss: 0.5562 - accuracy: 0.6480 - val_loss: 0.5127 - val_accuracy: 0.7634
Epoch 3/10
32/32 [==============================] - 34s 960ms/step - loss: 0.5149 - accuracy: 0.7200 - val_loss: 0.4880 - val_accuracy: 0.7527
Epoch 4/10
32/32 [==============================] - 35s 967ms/step - loss: 0.4756 - accuracy: 0.7360 - val_loss: 0.4521 - val_accuracy: 0.8065
Epoch 5/10
32/32 [==============================] - 36s 1s/step - loss: 0.4433 - accuracy: 0.7860 - val_loss: 0.4570 - val_accuracy: 0.7957
Epoch 6/10
32/32 [==============================] - 35s 981ms/step - loss: 0.4070 - accuracy: 0.8080 - val_loss: 0.4178 - val_accuracy: 0.8280
Epoch 7/10
32/32 [==============================] - 36s 1s/step - loss: 0.4147 - accuracy: 0.8100 - val_loss: 0.4386 - val_accuracy: 0.8280
Epoch

visualize

In [90]:
#visualize training results using tensorboard
%load_ext tensorboard
%tensorboard --logdir logs/fit


: 

: 